# Assignment 2: Entity Resolution (Part 1)

## Objective

Data scientists often spend 80% of their time on [data preparation](https://www.infoworld.com/article/3228245/the-80-20-data-science-dilemma.html). If your career goal is to become a data scientist, you have to master data cleaning and data integration skills. In this assignment, you will learn how to solve the Entity Resolution (ER) problem, a very common problem in data cleaning and integration. After completing this assignment, you should be able to answer the following questions:

1. What is ER?
2. What are the applications of ER in data integration and cleaning? 
3. How to avoid $n^2$ comparisons? 
4. How to compute Jaccard Similarity?
5. How to evaluate an ER result?

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

The data for Assignment 2 (Part 1 and Part 2) can be downloaded from [A2-data.zip](A2-data.zip).

## Overview

ER is defined as finding different records that refer to the same real-world entity, e.g., iPhone 4-th generation vs. iPhone four. It is central to data integration and cleaning. In this assignment, you will learn how to apply ER in a data integration setting. But the program that you are going to write can be easily extended to a data-cleaning setting, being used to detect _duplication values_.   

Imagine that you want to help your company's customers to buy products at a cheaper price. In order to do so, you first write a [web scraper](https://nbviewer.jupyter.org/github/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1-1.ipynb) to crawl product data from Amazon.com and Google Shopping, respectively, and then integrate the data together. Since the same product may have different representations in the two websites, you are facing an ER problem. 

Existing ER techniques can be broadly divided into two categories: similarity-based (Part 1) and learning-based (Part 2). 


## Similarity Join

Unlike a learning-based technique, a similarity-based technique (a.k.a similarity join) does not need any label data. It first chooses a similarity function and a threshold, and then returns the record pairs whose similarity values are above the threshold. These returned record pairs are thought of as matching pairs, i.e., referring to the same real-world entity. 

Depending on particular applications, you may need to choose different similarity functions. In this assignment, we will use Jaccard similarity, i.e., $\textsf{Jaccard}(r, s) = \big|\frac{r \cap s}{r \cup s}\big|$. Here is the formal definition of this problem.

> **Jaccard-Similarity Join**: Given two DataFrames, R and S, and a threshold $\theta \in (0, 1]$, the jaccard-similarity join problem aims to find all record pairs $(r, s) \in R \times S$ such that $\textsf{Jaccard}(r, s) \geq \theta$  

To implement similarity join, you need to address the following challenges:

1. Jaccard is used to quantify the similarity between two sets instead of two records. You need to convert each record to a set.

2. A naive implementation of similarity join is to compute Jaccard for all $|R \times S|$ possible pairs. Imagine R and S have one million records. This requires doing 10^12 pair comparisons, which is extremely expensive. Thus, you need to know how to avoid n^2 comparisons. 

3. The output of ER is a set of matching pairs, where each pair is considered as referring to the same real-world entity. You need to know how to evaluate the quality of an ER result.

Next, you will be guided to complete four tasks. After finishing these tasks, I suggest you going over the above challenges again, and understand how they are addressed.

Read the code first, and then implement the remaining four functions: `preprocess_df`, `filtering`, `verification`, and `evaluate` by doing Tasks A-D, respectively.

``` python
# similarity_join.py
import re
import pandas as pd

class SimilarityJoin:
    def __init__(self, data_file1, data_file2):
        self.df1 = pd.read_csv(data_file1)
        self.df2 = pd.read_csv(data_file2)
          
    def preprocess_df(self, df, cols): 
        """
            Write your code!
        """ 
    
    def filtering(self, df1, df2):
        """
            Write your code!
        """
      
    def verification(self, cand_df, threshold):
        """
            Write your code!
        """
        
    def evaluate(self, result, ground_truth):
        """
            Write your code!
        """
        
    def jaccard_join(self, cols1, cols2, threshold):
        new_df1 = self.preprocess_df(self.df1, cols1)
        new_df2 = self.preprocess_df(self.df2, cols2)
        print ("Before filtering: %d pairs in total" %(self.df1.shape[0] *self.df2.shape[0])) 
        
        cand_df = self.filtering(new_df1, new_df2)
        print ("After Filtering: %d pairs left" %(cand_df.shape[0]))
        
        result_df = self.verification(cand_df, threshold)
        print ("After Verification: %d similar pairs" %(result_df.shape[0]))
        
        return result_df
       
        

if __name__ == "__main__":
    er = SimilarityJoin("Amazon_sample.csv", "Google_sample.csv")
    amazon_cols = ["title", "manufacturer"]
    google_cols = ["name", "manufacturer"]
    result_df = er.jaccard_join(amazon_cols, google_cols, 0.5)

    result = result_df[['id1', 'id2']].values.tolist()
    ground_truth = pd.read_csv("Amazon_Google_perfectMapping_sample.csv").values.tolist()
    print ("(precision, recall, fmeasure) = ", er.evaluate(result, ground_truth))
```


The program will output the following when running on the sample data:


> Before filtering: 256 pairs in total

> After Filtering: 84 pairs left

> After Verification: 6 similar pairs

> (precision, recall, fmeasure) =  (1.0, 0.375, 0.5454545454545454)


In [420]:
import re
import pandas as pd

class SimilarityJoin:
    def __init__(self, data_file1, data_file2):
        self.df1 = pd.read_csv(data_file1)
        self.df2 = pd.read_csv(data_file2)

    def preprocess_df(self, df, cols): 
        df["joinKey"] = df[cols[0]].fillna('') + " " + df[cols[1]].fillna('')
        df['joinKey'] = df['joinKey'].str.lower()
        df["joinKey"] = df['joinKey'].str.split(r'\W+')

        return df

    def filtering(self, df1, df2):
        original_df1 = df1
        original_df2 = df2

        df1 = df1.explode('joinKey')
        df1 = df1[["id", "joinKey"]]
        df1['joinKey'].replace('', np.nan, inplace=True)
        df1.dropna(subset=['joinKey'], inplace=True)

        df2 = df2.explode('joinKey')
        df2 = df2[["id", "joinKey"]]
        df2['joinKey'].replace('', np.nan, inplace=True)
        df2.dropna(subset=['joinKey'], inplace=True)

        merge_result = df1.merge(df2, on="joinKey", suffixes=('_1', '_2'))
        merge_result = merge_result.drop_duplicates(subset=['id_1', 'id_2'], keep='first')

        res_1 = merge_result.merge(original_df1, left_on="id_1", right_on="id", suffixes=(None, '_1'))
        res_1 = res_1[["id_1", "joinKey_1", "id_2"]]

        res_2 = res_1.merge(original_df2, left_on="id_2", right_on="id")
        res_2 = res_2[["id_1", "joinKey_1", "id_2", "joinKey"]]
        res_2 = res_2.rename(columns={"joinKey": "joinKey_2"})
        return res_2

    def verification(self, cand_df, threshold):
        joinKey_1 = cand_df['joinKey_1']
        joinKey_2 = cand_df['joinKey_2']
        cand_df['jaccard'] = self.helper(joinKey_1, joinKey_2)
        final_df = cand_df.loc[(cand_df['jaccard'] >= threshold)]
        return final_df
    
    def helper(self, joinKey_1, joinKey_2):
        keys_1 = joinKey_1.values.tolist()
        keys_2 = joinKey_2.values.tolist()

        n = len(keys_1)
        res_list = []
        for i in range(n):
            # remove empty string
            new_keys_1 = list(filter(None, keys_1[i]))
            new_keys_2 = list(filter(None, keys_2[i]))
            new_keys_list = list(set(new_keys_1 + new_keys_2))
            dict = {}
            count = 0
            visited = set()
            for key in new_keys_1:
                dict[key] = 1
            for key in new_keys_2:
                if key in dict and key not in visited:
                    count += 1
                    visited.add(key)
            jaccard = count / len(new_keys_list)
            res_list.append(jaccard)
        sentence_series = pd.Series(res_list)
        return sentence_series

    def evaluate(self, result, ground_truth):
        true_match = 0
        for record in result:
            if record in ground_truth:
                true_match += 1
        precision = true_match / len(result)
        recall = true_match / len(ground_truth)
        fmeasure = (2 * precision * recall) / (precision + recall)
        return (precision, recall, fmeasure)

    def jaccard_join(self, cols1, cols2, threshold):
        new_df1 = self.preprocess_df(self.df1, cols1)
        new_df2 = self.preprocess_df(self.df2, cols2)
        print ("Before filtering: %d pairs in total" %(self.df1.shape[0] *self.df2.shape[0])) 

        cand_df = self.filtering(new_df1, new_df2)
        print ("After Filtering: %d pairs left" %(cand_df.shape[0]))

        result_df = self.verification(cand_df, threshold)
        print ("After Verification: %d similar pairs" %(result_df.shape[0]))

        return result_df



if __name__ == "__main__":
    er = SimilarityJoin("A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_sample.csv", "A2-data/part1-similarity-join/Amazon-Google-Sample/Google_sample.csv")
    amazon_cols = ["title", "manufacturer"]
    google_cols = ["name", "manufacturer"]
    result_df = er.jaccard_join(amazon_cols, google_cols, 0.5)

    result = result_df[['id_1', 'id_2']].values.tolist()
    ground_truth = pd.read_csv("A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_Google_perfectMapping_sample.csv").values.tolist()
    print ("(precision, recall, fmeasure) = ", er.evaluate(result, ground_truth))

Before filtering: 256 pairs in total
After Filtering: 84 pairs left
After Verification: 6 similar pairs
(precision, recall, fmeasure) =  (1.0, 0.375, 0.5454545454545454)


### Task A. Data Preprocessing (Record --> Token Set)

Since Jaccard needs to take two sets as input, your first job is to preprocess DataFrames by transforming each record into a set of tokens. Please implement the following function.   

```python
def preprocess_df(self, df, cols): 
    """ 
        Input: $df represents a DataFrame
               $cols represents the list of columns (in $df) that will be concatenated and be tokenized

        Output: Return a new DataFrame that adds the "joinKey" column to the input $df

        Comments: The "joinKey" column is a list of tokens, which is generated as follows:
                 (1) concatenate the $cols in $df; 
                 (2) apply the tokenizer to the concatenated string
        Here is how the tokenizer should work:
                 (1) Use "re.split(r'\W+', string)" to split a string into a set of tokens
                 (2) Convert each token to its lower-case
    """
    data = []
    for col in cols:
        print(df[col])
        
```

In [289]:
def preprocess_df(df, cols): 

    df["joinKey"] = df[cols[0]].fillna('') + " " + df[cols[1]].fillna('')
    df['joinKey'] = df['joinKey'].str.lower()
    df["joinKey"] = df['joinKey'].str.split(r'\W+')

    return df

        

In [291]:
import re
import pandas as pd
import numpy as np
df1 = pd.read_csv("A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_sample.csv")
amazon_cols = ["title", "manufacturer"]
new_df1 = preprocess_df(df1, amazon_cols)
new_df1
# new_df1.to_csv("amazon-test1.csv")

,id,title,description,manufacturer,price,joinKey
0,b0002mg5uk,iview mediapro 2.5,NaN,global marketing,199.99,"[iview, mediapro, 2, 5, global, marketing]"
1,b0002jtvng,bias deck le 3.5 macintosh cd,if you want to record music and audio like a p...,bias,99.00,"[bias, deck, le, 3, 5, macintosh, cd, bias]"
2,b0007lw22g,apple ilife '06 (mac dvd) [older version],ilife '06 is the easiest way to make the most ...,apple computer,79.00,"[apple, ilife, 06, mac, dvd, older, version, a..."
3,b00007kh02,extensis intellihance pro 4.x win/mac,extensis intellihance pro 4 quickly and dynami...,extensis corporation,199.99,"[extensis, intellihance, pro, 4, x, win, mac, ..."
4,b000saufpw,dk amazing animals 1.1,meet your cd host henry a delightful 3-d anima...,global software publishing,9.99,"[dk, amazing, animals, 1, 1, global, software,..."
5,b000v7vz1u,onone essentials for adobe photoshop elements ...,NaN,onone software,59.95,"[onone, essentials, for, adobe, photoshop, ele..."
6,b00006hvvo,upg sgms 1000 incremental node,today enterprises and service providers face i...,sonic systems inc.,0.00,"[upg, sgms, 1000, incremental, node, sonic, sy..."
7,b0000vyk1o,power director 3,powerdirector 3 - it's everything you need to ...,avanquest publishing usa inc.,79.95,"[power, director, 3, avanquest, publishing, us..."
8,b000jx1kma,aircraft power pack for ms flight simulator,aircraft powerpack includes wings of power: he...,red mile,29.99,"[aircraft, power, pack, for, ms, flight, simul..."
9,b000licg1m,power production storyboard artist 4,with storyboard artist 4 you can create profes...,power production,0.00,"[power, production, storyboard, artist, 4, pow..."


In [226]:
df2 = pd.read_csv("A2-data/part1-similarity-join/Amazon-Google-Sample/Google_sample.csv")
google_cols = ["name", "manufacturer"]
new_df2 = preprocess_df(df2, google_cols)
new_df2

,id,name,description,manufacturer,price,joinKey
0,http://www.google.com/base/feeds/snippets/1102...,punch software 20100 punch! 5 in 1 home design,total interior and exterior home design with r...,punch software,35.99,"[punch, software, 20100, punch, 5, in, 1, home..."
1,http://www.google.com/base/feeds/snippets/7249...,onone software ice-10770 intellihance pro 4.1 ...,dynamic image enhancement and color correction...,onone software,158.99,"[onone, software, ice, 10770, intellihance, pr..."
2,http://www.google.com/base/feeds/snippets/1227...,adobe cs3 web standard,system requirements powerpc g4 or g5 or intel ...,NaN,1035.99,"[adobe, cs3, web, standard, ]"
3,http://www.google.com/base/feeds/snippets/1835...,red mile entertainment 124 - aircraft power pa...,red mile entertainment 124 : with firepower fo...,NaN,27.97,"[red, mile, entertainment, 124, aircraft, powe..."
4,http://www.google.com/base/feeds/snippets/3409...,sonicwall gms 1000n-incremental lic upg,today enterprises and service providers face i...,NaN,62920.89,"[sonicwall, gms, 1000n, incremental, lic, upg, ]"
5,http://www.google.com/base/feeds/snippets/4377...,hijack2 identity and data security suite,prevent thieves from accessing any of your per...,NaN,44.90,"[hijack2, identity, and, data, security, suite, ]"
6,http://www.google.com/base/feeds/snippets/5572...,bias deck 3.5 le digital audio recording softw...,deck 3.5 le multi-track digital audio recordin...,NaN,69.95,"[bias, deck, 3, 5, le, digital, audio, recordi..."
7,http://www.google.com/base/feeds/snippets/1267...,iview mediapro 2.6 media management,product datasheet (pdf: 1.1mb) system requirem...,NaN,155.99,"[iview, mediapro, 2, 6, media, management, ]"
8,http://www.google.com/base/feeds/snippets/1084...,apple ilife '06,system requirements * macintosh computer with ...,NaN,79.99,"[apple, ilife, 06, ]"
9,http://www.google.com/base/feeds/snippets/1761...,onone software essentials for adobe photoshop ...,NaN,NaN,59.33,"[onone, software, essentials, for, adobe, phot..."


For the purpose of testing, you can compare your outputs with new_df1 and new_df2 that can be found from the `Amazon-Google-Sample` folder.

### Task B. Filtering Obviously Non-matching Pairs

To avoid $n^2$ pair comparisons, ER algorithms often follow a filtering-and-verification framework. The basic idea is to first filter obviously non-matching pairs and then only verify the remaining pairs.  

In Task B, your job is to implement the <font color="blue">filtering</font> function. This function will filter all the record pairs whose joinKeys do not share any token. This is because based on the definition of Jaccard, we can deduce that **if two sets do not share any element (i.e., $r\cap s = \phi$), their Jaccard similarity values must be zero**. Thus, we can safely remove them. 

```python
def filtering(self, df1, df2):
    """ 
        Input: $df1 and $df2 are two input DataFrames, where each of them 
               has a 'joinKey' column added by the preprocess_df function

        Output: Return a new DataFrame $cand_df with four columns: 'id1', 'joinKey1', 'id2', 'joinKey2',
                where 'id1' and 'joinKey1' are from $df1, and 'id2' and 'joinKey2'are from $df2.
                Intuitively, $cand_df is the joined result between $df1 and $df2 on the condition that 
                their joinKeys share at least one token. 

        Comments: Since the goal of the "filtering" function is to avoid n^2 pair comparisons, 
                  you are NOT allowed to compute a cartesian join between $df1 and $df2 in the function. 
                  Please come up with a more efficient algorithm (see hints in Lecture 2). 
    """
```

In [261]:
def filtering(df1, df2):
    original_df1 = df1
    original_df2 = df2
    
    df1 = df1.explode('joinKey')
    df1 = df1[["id", "joinKey"]]
    df1['joinKey'].replace('', np.nan, inplace=True)
    df1.dropna(subset=['joinKey'], inplace=True)
    
    df2 = df2.explode('joinKey')
    df2 = df2[["id", "joinKey"]]
    df2['joinKey'].replace('', np.nan, inplace=True)
    df2.dropna(subset=['joinKey'], inplace=True)

    merge_result = df1.merge(df2, on="joinKey", suffixes=('_1', '_2'))
    merge_result = merge_result.drop_duplicates(subset=['id_1', 'id_2'], keep='first')
    
    res_1 = merge_result.merge(original_df1, left_on="id_1", right_on="id", suffixes=(None, '_1'))
    res_1 = res_1[["id_1", "joinKey_1", "id_2"]]
    
    res_2 = res_1.merge(original_df2, left_on="id_2", right_on="id")
    res_2 = res_2[["id_1", "joinKey_1", "id_2", "joinKey"]]
    res_2 = res_2.rename(columns={"joinKey": "joinKey_2"})
    return res_2
    

In [262]:
cand_df = filtering(new_df1, new_df2)
cand_df

,id_1,joinKey_1,id_2,joinKey_2
0,b0002mg5uk,"[iview, mediapro, 2, 5, global, marketing]",http://www.google.com/base/feeds/snippets/1267...,"[iview, mediapro, 2, 6, media, management, ]"
1,b000v7vz1u,"[onone, essentials, for, adobe, photoshop, ele...",http://www.google.com/base/feeds/snippets/1267...,"[iview, mediapro, 2, 6, media, management, ]"
2,b0002mg5uk,"[iview, mediapro, 2, 5, global, marketing]",http://www.google.com/base/feeds/snippets/1761...,"[onone, software, essentials, for, adobe, phot..."
3,b000v7vz1u,"[onone, essentials, for, adobe, photoshop, ele...",http://www.google.com/base/feeds/snippets/1761...,"[onone, software, essentials, for, adobe, phot..."
4,b00002s6sc,"[punch, 5, in, 1, home, design, punch, software]",http://www.google.com/base/feeds/snippets/1761...,"[onone, software, essentials, for, adobe, phot..."
...,...,...,...,...
79,b000ndibge,"[adobe, creative, suite, cs3, web, standard, m...",http://www.google.com/base/feeds/snippets/4377...,"[hijack2, identity, and, data, security, suite, ]"
80,b000in8n30,"[hijack2, navarre, software, ]",http://www.google.com/base/feeds/snippets/4377...,"[hijack2, identity, and, data, security, suite, ]"
81,b00013wh0w,"[handmark, oxford, american, desk, dictionary,...",http://www.google.com/base/feeds/snippets/4377...,"[hijack2, identity, and, data, security, suite, ]"
82,b000gaqlxe,"[aquazone, bass, edition, smith, micro, software]",http://www.google.com/base/feeds/snippets/1479...,"[allume, aquazone, bass, edition, ]"


For the purpose of testing, you can compare your output with cand_df that can be found from the `Amazon-Google-Sample` folder.

### Task C. Computing Jaccard Similarity for Survived Pairs

In the second phase of the filtering-and-verification framework, we will compute the Jaccard similarity for each survived pair and return those pairs whose jaccard similarity values are no smaller than the specified threshold.

In Task C, your job is to implement the <font color="blue">verification</font> function. This task looks simple, but there are a few small "traps". 


```python
def verification(self, cand_df, threshold):
        """ 
            Input: $cand_df is the output DataFrame from the 'filtering' function. 
                   $threshold is a float value between (0, 1] 

            Output: Return a new DataFrame $result_df that represents the ER result. 
                    It has five columns: id1, joinKey1, id2, joinKey2, jaccard 

            Comments: There are two differences between $cand_df and $result_df
                      (1) $result_df adds a new column, called jaccard, which stores the jaccard similarity 
                          between $joinKey1 and $joinKey2
                      (2) $result_df removes the rows whose jaccard similarity is smaller than $threshold 
        """
```

In [396]:
def verification(cand_df, threshold):
    joinKey_1 = cand_df['joinKey_1']
    joinKey_2 = cand_df['joinKey_2']
    cand_df['jaccard'] = helper(joinKey_1, joinKey_2, threshold)
    final_df = cand_df.loc[(cand_df['jaccard'] >= threshold)]
    return final_df

In [397]:
def helper(joinKey_1, joinKey_2):
    keys_1 = joinKey_1.values.tolist()
    keys_2 = joinKey_2.values.tolist()

    n = len(keys_1)
    res_list = []
    for i in range(n):
        # remove empty string
        new_keys_1 = list(filter(None, keys_1[i]))
        new_keys_2 = list(filter(None, keys_2[i]))
        new_keys_list = list(set(new_keys_1 + new_keys_2))
        dict = {}
        count = 0
        visited = set()
        for key in new_keys_1:
            dict[key] = 1
        for key in new_keys_2:
            if key in dict and key not in visited:
                count += 1
                visited.add(key)
        jaccard = count / len(new_keys_list)
        res_list.append(jaccard)
    sentence_series = pd.Series(res_list)
    return sentence_series
        

In [401]:
result_df = verification(cand_df, 0.5)
result_df

,id_1,joinKey_1,id_2,joinKey_2,jaccard
3,b000v7vz1u,"[onone, essentials, for, adobe, photoshop, ele...",http://www.google.com/base/feeds/snippets/1761...,"[onone, software, essentials, for, adobe, phot...",1.000000
15,b00002s6sc,"[punch, 5, in, 1, home, design, punch, software]",http://www.google.com/base/feeds/snippets/1102...,"[punch, software, 20100, punch, 5, in, 1, home...",0.875000
56,b000licg1m,"[power, production, storyboard, artist, 4, pow...",http://www.google.com/base/feeds/snippets/6070...,"[power, production, power, production, storybo...",0.571429
64,b000jx1kma,"[aircraft, power, pack, for, ms, flight, simul...",http://www.google.com/base/feeds/snippets/1835...,"[red, mile, entertainment, 124, aircraft, powe...",0.529412
68,b000ndibbo,"[adobe, indesign, cs3, upgrade, from, pagemake...",http://www.google.com/base/feeds/snippets/1693...,"[adobe, indesign, cs3, for, mac, upgrade, from...",0.875000
73,b000ndibge,"[adobe, creative, suite, cs3, web, standard, m...",http://www.google.com/base/feeds/snippets/1227...,"[adobe, cs3, web, standard, ]",0.571429


For the purpose of testing, you can compare your output with result_df that can be found from the `Amazon-Google-Sample` folder.

### Task D. Evaluating an ER result

How should we evaluate an ER result? Before answering this question, let's first recall what the ER result looks like. The goal of ER is to identify all matching record pairs. Thus, the ER result should be a set of identified matching pairs, denoted by R. One thing that we want to know is that what percentage of the pairs in $R$ that are truly matching? This is what Precision can tell us. Let $T$ denote the truly matching pairs in $R$. Precision is defined as:
$$Precision = \frac{|T|}{|R|}$$

In addition to Precision, another thing that we care about is that what percentage of truly matching pairs that are identified. This is what Recall can tell us. Let $A$ denote the truly matching pairs in the entire dataset. Recall is defined as: 

$$Recall = \frac{|T|}{|A|}$$

There is an interesting trade-off between Precision and Recall. As more and more pairs that are identified as matching, Recall increases while Precision potentially decreases. For the extreme case, if we return all the pairs as matching pairs, we will get a perfect Recall (i.e., Recall = 100%) but precision will be the worst. Thus, to balance Precision and Recall, people often use FMeasure to evaluate an ER result:

$$FMeasure = \frac{2*Precision*Recall}{Precision+Recall}$$

In Task D, you will be given an ER result as well as the ground truth that tells you what pairs are truly matching. Your job is to calculate Precision, Recall and FMeasure for the result. 
```python
def evaluate(self, result, ground_truth):
    """ 
        Input: $result is a list of matching pairs identified by the ER algorithm
               $ground_truth is a list of matching pairs labeld by humans

        Output: Compute precision, recall, and fmeasure of $result based on $ground_truth, and
                return the evaluation result as a triple: (precision, recall, fmeasure)

    """
```


In [414]:
result = result_df[['id_1', 'id_2']].values.tolist()
ground_truth = pd.read_csv("A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_Google_perfectMapping_sample.csv").values.tolist()
print ("(precision, recall, fmeasure) = ", evaluate(result, ground_truth))



(precision, recall, fmeasure) =  (1.0, 0.375, 0.5454545454545454)


In [412]:
def evaluate(result, ground_truth):
    true_match = 0
    for record in result:
        if record in ground_truth:
            true_match += 1
    precision = true_match / len(result)
    recall = true_match / len(ground_truth)
    fmeasure = (2 * precision * recall) / (precision + recall)
    return (precision, recall, fmeasure)
    

## Submission

Implement `preprocess_df`, `filtering`, `verification`, and `evaluate` functions in `similarity_join.py`. Submit your code file (`similarity_join.py`)  to the CourSys activity Assignment 2.